In [ ]:
# Unit Test for Fix Gen Fun


# Mock function to simulate generating a list of dates
def mock_generate_date_list(reference, base_date, func_type):
    # Dictionary to map reference to the number of months to shift
    date_offsets = {
        '01-': -12, '02-': -11, '03-': -10, '04+': 4, '04-': -9, '05-': -7, '06-': -6,
        '07-': -5, '08-': -4, '09+': 9, '09-': -3, '10-': -2, '11-': -1, '12-': 0
    }
    # Calculate shifted dates based on the base date and date offsets
    shifted_dates = [(base_date + dateutil.relativedelta.relativedelta(months=date_offsets.get(ref, 0))).strftime('%Y%m') for ref in reference.split(':')]
    # Return a DataFrame containing the shifted dates
    return pd.DataFrame({'date': [int(date) for date in shifted_dates]})

# Mock function to simulate calculating the relative increase
def mock_calculate_relative_increase(new_value, base_value, index_value):
    try:
        base_value = float(base_value)
    except ValueError:
        base_value = 0
    if base_value == 0:
        return new_value
    return (new_value / base_value) * float(index_value)

# The function to be tested
def func_generate_fix(premises, reference, parameters={}):
    function = 'FIX'  # Define the function type as 'FIX'
    month_interval = VARIATENINTERVAL  # Use the global variable VARIATENINTERVAL to determine the month interval
    start_date = datetime.strptime(STARTDATE, '%d.%m.%Y')  # Convert the start date from string to datetime object
    end_date = datetime.strptime(ENDDATE, '%d.%m.%Y')  # Convert the end date from string to datetime object

    df_result = pd.DataFrame(columns=['date', 'Index_Value', 'premise_values'])  # Initialize an empty DataFrame to store the results
    current_date = start_date  # Set the current date to the start date

    relative_date = datetime.strptime(str(parameters['Base_Date']), '%Y%m')  # Convert the base date from the parameters to a datetime object

    while current_date <= end_date:  # Loop until the current date is less than or equal to the end date
        # Generate a list of dates for the current date
        df_datelist = mock_generate_date_list(reference, current_date, function).date.astype('int64')
        # Merge the premises DataFrame with the generated date list
        df_calc = premises.merge(df_datelist, how='right', on='date')
        # Calculate the mean of the premise values for the target date
        targetdate_value = df_calc.premise.mean()

        # Generate a list of dates for the relative date
        rdf_datelist = mock_generate_date_list(reference, relative_date, function).date.astype('int64')
        # Merge the premises DataFrame with the relative date list
        rdf_calc = premises.merge(rdf_datelist, how='right', on='date')
        # Calculate the mean of the premise values for the relative date
        relativedate_value = rdf_calc.premise.mean()

        # Calculate the relative increase
        calc_value = mock_calculate_relative_increase(targetdate_value, relativedate_value, parameters['Index_Value'])

        # If debug mode is enabled in parameters, print debug information
        if '_debug' in parameters and parameters['_debug']:
            df_debug = df_calc.merge(rdf_calc.rename(columns={'premise': 'ref_premise'}), how='outer', on='date')
            print(df_debug)
            print(f'date: {current_date} value: {calc_value} premise.mean(): {targetdate_value} ref_premise.mean(): {relativedate_value}')

        # Create a string with the premise values and index value
        premise_str = f'new_value:{targetdate_value} base_value:{relativedate_value} index_value:{parameters["Index_Value"]}'
        # Append the calculated values to the result DataFrame
        df_result = df_result.append({'date': current_date.strftime('%Y%m'), 'Index_Value': calc_value, 'premise_values': premise_str}, ignore_index=True)
        # Advance the current date by the month interval
        current_date += dateutil.relativedelta.relativedelta(months=month_interval)

    return df_result  # Return the result DataFrame

# Test case class
class TestFuncGenerateFix(unittest.TestCase):
    @patch('__main__.mock_generate_date_list', side_effect=mock_generate_date_list)
    @patch('__main__.mock_calculate_relative_increase', side_effect=mock_calculate_relative_increase)
    def test_func_generate_fix(self, mock_generate_date_list, mock_calculate_relative_increase):
        # Sample data for the premises DataFrame
        sample_data = {
            'date': [202201, 202202, 202203, 202204, 202205, 202206, 202207, 202208, 202209, 202210, 202211, 202212],
            'premise': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120]
        }
        premises = pd.DataFrame(sample_data)  # Create the premises DataFrame
        reference_dates = '02-:03-:04-:05-:06-:07-:08-:09-:10-:11-:12-'  # Define the reference dates
        params = {
            'Index_Value': '177.6',  # Set the index value
            'Base_Date': '202301',  # Set the base date
            '_debug': True  # Enable debug mode
        }

        global VARIATENINTERVAL, STARTDATE, ENDDATE
        VARIATENINTERVAL = 1  # Set the month interval to 1
        STARTDATE = '01.01.2022'  # Set the start date
        ENDDATE = '01.12.2022'  # Set the end date

        # Expected output DataFrame
        expected_output = pd.DataFrame({
            'date': ['202201', '202202', '202203', '202204', '202205', '202206', '202207', '202208', '202209', '202210', '202211', '202212'],
            'Index_Value': [24.666666666666668, 37.0, 49.333333333333336, 61.666666666666664, 74.0, 86.33333333333333, 98.66666666666667, 111.0, 135.66666666666666, 145.25925925925924, 155.4, 165.9393939393939],
            'premise_values': [
                'new_value:10.0 base_value:72.0 index_value:177.6',
                'new_value:15.0 base_value:72.0 index_value:177.6',
                'new_value:20.0 base_value:72.0 index_value:177.6',
                'new_value:25.0 base_value:72.0 index_value:177.6',
                'new_value:30.0 base_value:72.0 index_value:177.6',
                'new_value:35.0 base_value:72.0 index_value:177.6',
                'new_value:40.0 base_value:72.0 index_value:177.6',
                'new_value:45.0 base_value:72.0 index_value:177.6',
                'new_value:55.0 base_value:72.0 index_value:177.6',
                'new_value:58.888888888888886 base_value:72.0 index_value:177.6',
                'new_value:63.0 base_value:72.0 index_value:177.6',
                'new_value:67.27272727272727 base_value:72.0 index_value:177.6'
            ]
        })

        result = func_generate_fix(premises, reference_dates, params)  # Call the function with the sample data and parameters

        # Assert that the result matches the expected output
        pd.testing.assert_frame_equal(result, expected_output)

if __name__ == '__main__':
    unittest.main(argv=[''], exit=False)  # Run the unit tests


In [ ]:
# LISTREL Unit test


VARIATENINTERVAL = 1  # Month interval
STARTDATE = '01.01.2023'
ENDDATE = '01.04.2023'

# Mock the external functions
def gen_dateliste_from_refstring_array(reference, current_date, function):
    # This function generates a mock DataFrame with 'datum' column
    dates = pd.date_range(current_date, periods=3, freq='M').astype('int64') // 10**9
    return pd.DataFrame({'datum': dates})

def calc_relative_increase(target, reference, index_value):
    # Simple mock calculation: return the difference
    return (target - reference) / reference * index_value

class TestFuncGenerateListrel(unittest.TestCase):
    def test_func_generate_listrel(self):
        # Mock data for praemissen DataFrame
        praemissen_data = {
            'datum': [1672444800, 1675036800, 1677628800],  # Mock dates in int64 format
            'praemisse': [100, 200, 300]
        }
        praemissen = pd.DataFrame(praemissen_data)

        # Mock reference and parameters
        reference = ['01', '02', '03']
        parameters = {
            'AP_Datum': '202301',
            'Index_Wert': 1.0,
            '_debug': False
        }

        # Call the function to test
        result = func_generate_listrel(praemissen, reference, parameters)

        # Check the result DataFrame
        self.assertEqual(len(result), 4)  # We expect 4 rows (January to April)
        self.assertEqual(list(result.columns), ['datum', 'Index_Wert', 'praemisse_values'])  # Check column names

        # Check some values in the result DataFrame
        self.assertEqual(result.iloc[0]['datum'], '202301')
        self.assertAlmostEqual(result.iloc[0]['Index_Wert'], 0.0)  # Since we mock simple calculation, expecting 0.0

if __name__ == '__main__':
    unittest.main()

